# Crawl Data Analysis: Clustering

This notebook tries various clustering techniques on our web crawl data. It was written for Python 2.7, and assumes it's running on cycles. You can view/edit the notebook remotely as follows:

- Clone the GitHub repo to cycles (e.g. spin.cs.princeton.edu)
- Start up this notebook. Jupyter is not installed globally, but you can install it locally with pip via `pip install --user jupyter`. Then you can run this notebook in a tmux session: `tmux`, then `cd [this directory]`, then `jupyter notebook --no-browser --port 8889` (note that you can choose whatever port number you want, but we'll assume from here on it's 8889). Copy the URL generated - this is the URL you'll visit in your browser to open the notebook. Then Ctrl-B, D to detach the tmux session, and log out of cycles.
- On your local machine, forward your local port 8889 to the remote port 8889 on cycles: `ssh -L 8889:localhost:8889 [netid]@spin.cs.princeton.edu`
- Now you can open the notebook in your browser by pasting the link you copied earlier.

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import os

## Read from database

Read the crawl data from the database. Here we read in the `site_visits` and `segments` tables and join them.

In [ ]:
import sqlite3
import pandas as pd

db = '/n/fs/darkpatterns/crawl/2018-12-08_segmentation_pilot2/2018-12-08_segmentation_pilot2.sqlite'
con = sqlite3.connect(db)
site_visits = pd.read_sql_query('''SELECT * from site_visits''', con)

In [ ]:
print('Number of site visits: %s' % str(site_visits.shape))
print('site_visits columns: %s' % str(list(site_visits.columns.values)))

Report how many unique domains we have.

In [ ]:
from urlparse import urlparse

site_visits['domain'] = site_visits['site_url'].apply(lambda x: urlparse(x).netloc)
grouped = site_visits.groupby(['domain']).count().sort_values('visit_id', ascending=False)

In [ ]:
print('Number of unique domains: %s' % str(grouped.shape[0]))

In [ ]:
segments = pd.read_sql_query('''SELECT * from segments''', con)

In [ ]:
segments = segments.reset_index().set_index('visit_id').join(site_visits.reset_index()[['visit_id', 'site_url', 'domain']].set_index('visit_id'), how='inner')

In [ ]:
print('Number of segments: %s' % str(segments.shape))
print('segments columns: %s' % str(list(segments.columns.values)))

## Preprocess data

Ignore `body` tags and null `inner_text`.

In [ ]:
segments['inner_text'] = segments['inner_text'].str.strip()
segments = segments[(segments['node_name'] != 'BODY') & (segments['inner_text'] != '')]

Add a new column for number of newlines in each segment.

In [ ]:
segments['newline_count'] = segments['inner_text'].apply(lambda x: len(x.split('\n')))

Apply some standard techniques in preprocessing string data (ref: [Kdnuggets article](https://www.kdnuggets.com/2017/06/text-clustering-unstructured-data.html)):

- Lower case
- Replacing numbers with a placeholder, and replace units of measure with a placeholder (e.g kg, sq ft, gb, etc.)
- Removing punctuation
- Removing excess whitespace
- Removing known "stop words"/"stop phrases"/generic words (articles, conjunctions, etc.)
- Stemming (reducing words to their stems, i.e. via Porter's stemming algorithm)

In [ ]:
from nltk.stem.porter import PorterStemmer
import nltk
import re
import string

nltk.download('stopwords')

stemmer = PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords.union(set(string.punctuation))

def preprocess(s):
    s = s.lower()
    s = re.sub(r'\d+', 'dpnum', s)
    s = re.sub(r'[^a-z\s]', '', s)
    s = re.sub(r'\s+', ' ', s)
    words = s.split()
    words = [stemmer.stem(w) for w in words if len(w) > 0 and w not in stopwords]
    
    # Optimization to get rid of suffixes (e.g. units of measure)
    for i in range(len(words)):
        if words[i].startswith('dpnum'):
            words[i] = 'dpnum'
    
    return ' '.join(words)

In [ ]:
# Sanity check
s = '''Color   
Choose an option
Silver
Space Gray

Size    
Choose an option
64 GB'''
print('ORIGINAL STRING:')
print(s)
print()
print('PREPROCESSED STRING:')
print(preprocess(s))

In [ ]:
segments['inner_text_processed'] = segments['inner_text'].apply(preprocess)
segments['longest_text_processed'] = segments['longest_text'].apply(preprocess)

Add new columns for length of original text and processed text.

In [ ]:
segments['inner_text_length'] = segments['inner_text'].apply(lambda x: len(x))
segments['inner_text_processed_length'] = segments['inner_text_processed'].apply(lambda x: len(x))
segments['longest_text_length'] = segments['longest_text'].apply(lambda x: len(x))
segments['longest_text_processed_length'] = segments['longest_text_processed'].apply(lambda x: len(x))

In [ ]:
new_cols = ['newline_count', 'inner_text_length', 'inner_text_processed_length',
            'longest_text_length', 'longest_text_processed_length']
for c in new_cols:
    print('segments[\'%s\'].describe():\n%s' % (c, segments[c].describe().to_string()))

Remove redundant segments.

In [ ]:
segments = segments.groupby(['domain']).apply(lambda x: x.drop_duplicates(subset=['inner_text_processed'], keep='last'))

In [ ]:
print('Number of segments: %s' % str(segments.shape))
print('segments columns: %s' % str(list(segments.columns.values)))

Select a random subset of the data to make clustering more tractable during trial-error.

In [ ]:
segments_orig = segments.copy(deep=False)
indices = np.random.choice(np.arange(segments.shape[0]), replace=False, size=25000)
segments = segments.iloc[indices,:]

In [ ]:
print('segments shape: %s' % str(segments.shape))

## Create feature vectors

First we define the a function to tokenize text as we convert text into feature vectors. 

In [ ]:
from nltk.stem.porter import PorterStemmer
import nltk

nltk.download('stopwords')

stemmer = PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

def tokenize(line):
    if (line is None):
        line = ''
    tokens = [stemmer.stem(t) for t in nltk.word_tokenize(line) if len(t) != 0 and t not in stopwords and not t.isdigit()]
    return tokens

Now select one of the following cells to run to create a feature representation. Either load from the pre-existing file or recompute the features.

### 1. Bag of words

In [ ]:
if os.path.isfile('output/features_bow.npy'):
    features = np.load('output/features_bow.npy')
    print('Loaded from file')
else:
    print('No pre-existing file')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

data = segments['inner_text_processed']
vec = CountVectorizer(tokenizer=tokenize, binary=binary_rep, strip_accents='ascii').fit(data)

In [ ]:
print('Length of vocabulary %s' % str(len(vec.vocabulary_)))

In [ ]:
vec = vec.transform(data)
features = normalize(vec, axis=0)
np.save('output/features_bow.npy', features)

### 2. TFIDF

Optionally load from a file (enable that cell and disable the others that compute the TFIDF features if you have a file present).

In [ ]:
if os.path.isfile('output/features_tfidf.npy'):
    features = np.load('output/features_tfidf.npy')
    print('Loaded from file')
else:
    print('No pre-existing file')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
data = segments['inner_text_processed']
vec = TfidfVectorizer(binary=False, strip_accents='ascii', max_features=1000).fit(data)
features = vec.transform(data)
np.save('output/features_tfidf.npy', features)

In [ ]:
print('features shape (num_examples, num_features): %s' % str(features.shape))
print('Length of vocabulary: %s' % str(len(vec.vocabulary_)))

In [ ]:
sorted_vocab = sorted([(t, f) for t, f in vec.vocabulary_.iteritems()], cmp=lambda x, y: x[1] - y[1])
print('Vocabulary')
print('\n'.join(['%d: %s' % (f, t) for t, f in sorted_vocab]))

### 3. Word Vectors

We compute a vector for each segment as follows: compute the word vector for each word in the segment's `inner_text`, and then average over all words in that segment.

While it's simple, there are clearly downsides to this approach:

- We lose information about word ordering
- All words are equally weighted, so words that really characterize the text are not prioritized

In [ ]:
if os.path.isfile('output/features_wordvec.npy'):
    features = np.load('output/features_wordvec.npy')
    print('Loaded from file')
else:
    print('No pre-existing file')
    import en_core_web_sm

    data = segments['inner_text_processed']
    nlp = en_core_web_sm.load()
    vecs = []
    for doc in nlp.pipe(data.str.replace(r'\d+', '').astype('unicode').values, batch_size=10000, n_threads=7):
        if doc.is_parsed:
            vecs.append(doc.vector)
        else:
            vecs.append(None)
    features = np.array(vecs)
    np.save('output/features_wordvec.npy', features)
    
print('features shape: %s' % str(features.shape))

### PCA
Try using PCA to reduce the dimension of the data.

The feature matrix is expected to be provided with examples in rows (`num_examples` x `num_features`).

Projected data is given by $U^T X$, where $U$ is matrix with PCs in columns (`orig_dim` x `reduced_dim`), and $X$ is the data matrix with examples in columns (`orig_dim` x `num_examples`). In our case, $U^T$ is `pca.components_` and $X$ is `features.T`.

In [ ]:
if os.path.isfile('output/features_proj.npy'):
    features = np.load('output/features_proj.npy')
    print('Loaded from file')
else:
    print('No pre-existing file')
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=5)
    # pca = PCA(tol=10)
    pca.fit(features)
    
    print('Matrix of PCs: %s' % str(pca.components_.shape))
    print('Data matrix: %s' % str(features.shape))
    print('%d singular values: %s' % (pca.singular_values_.shape[0], str(pca.singular_values_)))
    
    features = np.dot(pca.components_, features.T)
    features = features.T
    
    np.save('output/features_proj.npy', features)
    
print('feature matrix shape (after PCA): %s' % str(features.shape))

Plot the data in the reduced dimension (assuming new dimension is 3, otherwise this is meaningless).

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features[:,0], features[:,1], features[:,2])

## Clustering

Run one of the following clustering algorithms.

### 1. Hierarchical clustering

In [ ]:
from scipy.spatial import distance
import fastcluster

# featdense = features.todense()
distances = distance.pdist(features, metric='cosine')
distances = distance.squareform(distances, checks=False)

In [ ]:
clusters = fastcluster.linkage(distances, method='ward', preserve_input=False)
np.save('output/hierarchical_linkage_matrix.npy', clusters)

Plot a dendogram of the resulting clusters.

In [ ]:
from scipy.cluster.hierarchy import dendrogram

plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    clusters,
    leaf_rotation=90.,
    leaf_font_size=8.,
)
plt.show()

### 2. DBSCAN clustering

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.externals.joblib.parallel import parallel_backend

with parallel_backend('threading'):
    clusterer = DBSCAN(eps=0.0001, min_samples=3, n_jobs=10, metric='euclidean')
    cluster_labels = clusterer.fit(features)

In [ ]:
segments['cluster_dbscan'] = pd.Series(cluster_labels.labels_).values

In [ ]:
print('Number of clusters: %d' % len(set(cluster_labels.labels_)))
print('segments[\'cluster_dbscan\'].value_counts(): \n %s' % segments['cluster_dbscan'].value_counts().to_string())

### 3. HDBSCAN clustering

In [ ]:
from sklearn.preprocessing import normalize
import hdbscan

features = normalize(features, axis=1) # Normalize each segment since using euclidean distance metric
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean')
cluster_labels = clusterer.fit_predict(features)
segments['cluster_hdbscan'] = pd.Series(cluster_labels).values

In [ ]:
print('segments[\'cluster_hdbscan\'].value_counts(): \n %s' % segments['cluster_hdbscan'].value_counts().to_string())

### 4. Expectation Maximization (EM)

Reference/inspiration found [here](https://suif.stanford.edu/~livshits/papers/pdf/uwpc.pdf) and [here](https://www.datascience.com/blog/k-means-alternatives).

The form of EM we use here can be thought of as a generalization of K-means clustering, using "soft" clusters instead of "hard" ones. In relation to K-means: at each iteration, rather than assign every data point to a single cluster, EM computes probability that the point belongs to each cluster, giving a list of probabilities for each data point. Then parameters are updated to maximize the probability of having this distribution of the data.

In [ ]:
from sklearn.mixture import GaussianMixture

em = GaussianMixture(n_components=100, max_iter=15, tol=0.05, random_state=13, verbose=2, verbose_interval=1)
em.fit(features.todense())

In [ ]:
cluster_labels = em.predict(features.todense())

In [ ]:
segments['cluster_em'] = pd.Series(cluster_labels).values

In [ ]:
np.save('output/em_cluster_labels.npy', cluster_labels)

In [ ]:
print('Number of clusters: %d' % len(set(cluster_labels)))
print('segments[\'cluster_em\'].value_counts(): \n %s' % segments['cluster_em'].value_counts().to_string())

## Evaluate Performance

Below we compute a few performance metrics on the clusters to get a sense of how well each method did.

### 1. Silhouette Coefficient

Ref: https://scikit-learn.org/stable/modules/clustering.html#silhouette-coefficient

Computes for each sample:

$$\frac{b - a}{\max(a, b)}$$

where $a$ is the mean distance between this point and other points in its assigned cluster, and $b$ is the mean distance between this point and other points in the next nearest cluster (based on a distance metric, which is a parameter, such as Euclidean distance).

**TL;DR** Ranges between -1 (poorly assigned labels) and +1 (highly dense and separated clusters). In particular, a score near 0 indicates overlapping clusters.

In [ ]:
from sklearn import metrics

s = metrics.silhouette_score(features, segments['cluster'], metric='euclidean')

In [ ]:
print('Silhouette score: %.3f' % s)

### 2. Calinski-Harabaz Index

Ref: https://scikit-learn.org/stable/modules/clustering.html#calinski-harabaz-index
    
Computes the ratio of between-cluster dispersion to within-cluster dispersion. "Between-cluster dispersion" is is computed by taking the weighted sum of "scatter matrices" between cluster centroids and the centroid of the entire dataset:
$$\sum_q n_q (c_q - c)(c_q - c)^T$$
where $n_q$ is the number of points in cluster $q$, $c_q$ is the centroid of cluster $q$, and $c$ is the center of all the data; and then taking the trace.

"Within-cluster dispersion" for a single cluster is an analogous measure using the scatter matrix over that cluster:
$$\sum_{x \in \text{cluster } q} (x - c_q)(x - c_q)^T$$
where $c$ is the cluster centroid. Total dispersion is computed by summing these scatter matrices over all clusters and taking the trace.

**TL;DR** Larger values are better and indicate denser, more separated clusters. Values near 0 indicate overlapping clusters. This is also the fastest metric to compute, since it's very fast to generate scatter matrices (as opposed to the other metrics, which must compute many pairwise distances).

In [ ]:
from sklearn import metrics
from scipy.sparse import issparse

if issparse(features):
    chi = metrics.calinski_harabaz_score(features.todense(), segments['cluster'])
else:
    chi = metrics.calinski_harabaz_score(features, segments['cluster'])

In [ ]:
print('Score: %.3f' % chi)

### 3Davies-Bouldin Index

Ref: https://scikit-learn.org/stable/modules/clustering.html#davies-bouldin-index

Computes the average "similarity" between each cluster and its most similar one. Similarity between clusters $i$ and $j$ is $R_{ij}$ and is computed as:
$$R_{ij} = \frac{s_i + s_j}{d_{ij}}$$
where $s_i$ is the average distance from $i$'s centroid to its points ("cluster diameter"), and $d_{ij}$ is the distance between cluster centroids. Then the DB index is the average:
$$\frac{1}{k}\sum_{i} \max_{j \neq i} R_{ij}$$

**TL;DR** Similarity values closer to 0 indicate better separation of clusters. Thus DB values closer to 0 are better, and larger values indicate sparse and/or overlapping clusters.

In [ ]:
from sklearn.metrics import davies_bouldin_score

dbi = davies_bouldin_score(features, segments['cluster'])

In [ ]:
print('Score: %.3f' % dbi)

## Visualize results

Produce a CSV file that shows the segments in each cluster.

In [ ]:
inner_texts = segments['inner_text']
cluster_labels = segments['cluster']
urls = segments['site_url']
print("segments['inner_text'] is %s, segments['cluster'] is %s, segments['site_url'] is %s (should be the same)" % (str(inner_texts.shape), str(cluster_labels.shape), str(urls.shape)))

Group the segments by cluster.

In [ ]:
from collections import defaultdict

inner_text_by_cluster = defaultdict(lambda: [])
url_by_cluster = defaultdict(lambda: [])
for i in range(inner_texts.shape[0]):
    inner_text_by_cluster[str(cluster_labels[i])].append(inner_texts[i])
    url_by_cluster[str(cluster_labels[i])].append(urls[i])

Write CSV file.

In [ ]:
import unicodecsv as csv
from datetime import datetime

timestamp = '_'.join(str(datetime.now()).split(' '))
outfile = 'output/clusters-%s.csv' % timestamp
with open(outfile, 'wb') as f:
    writer = csv.writer(f)
    for cluster in inner_text_by_cluster.keys():
        segments_str = '\n\n'.join(['%s:\n%s' % (u, t) for u, t in zip(url_by_cluster[cluster], inner_text_by_cluster[cluster])])
        writer.writerow([cluster, segments_str, url_by_cluster[cluster]])

Print some clusters.

In [ ]:
sorted_clusters = sorted([(c, texts) for c, texts in inner_text_by_cluster.iteritems()], cmp=lambda x, y: len(y[1]) - len(x[1]))

In [ ]:
c, texts = sorted_clusters[54]
urls = url_by_cluster[c]
print('Cluster %s (length %d)' % (c, len(texts)))
print('----------')
print('\n\n'.join(['%s:\n%s' % (u, t) for u, t in zip(urls, texts)]))
print()